In [1]:
import os, re, copy
from pathlib import Path
import pandas as pd
import requests 
from stop_words import get_stop_words
import warnings 
from pathlib import Path
from bs4 import BeautifulSoup 
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [2]:
import os
import re
from pprint import pprint
import argparse
import ast
from pathlib import Path
import pathlib
import re
import pprint 
import argparse

p = Path('.')

json_string='''@article{la1996expectations,
  title={Expectations and the cross-section of stock returns},
  author={La Porta, Rafael},
  journal={The Journal of Finance},
  volume={51},
  number={5},
  pages={1715--1742},
  year={1996},
  publisher={Wiley Online Library}
}'''

url= f'https://scholar.google.com/scholar?cites=15845218875244839334&as_sdt=5,47&sciodt=0,47&hl=en' # user manual input

collect_dict_values= {}
for item in json_string.split('\n'):
    ii= item.split('=')
    if 'year' in [i.strip() for i in ii ]:
        collect_dict_values['year']=re.sub('\W+','-', item.split('=')[1])
    if 'title' in [i.strip() for i in ii ]:
        collect_dict_values['title']=re.sub('\W+','-', item.split('=')[1])
    if 'author' in [i.strip() for i in ii ]:
        collect_dict_values['author']=re.sub('\W+','-', item.split('=')[1])
        # print(re.sub('\W+','-', item.split('=')[1]))
    if 'journal' in [i.strip() for i in ii ]:
        collect_dict_values['journal']=re.sub('\W+','-', item.split('=')[1])
        
print(''.join(('article-',collect_dict_values['year'],collect_dict_values['author'],
               collect_dict_values['title'], collect_dict_values['journal'][:-1],'.pdf',)))
citation_key= [item.split(',') for item in json_string.split('\n')][0][0].split('{')[1]

print('CITATION KEY: ', citation_key)

citation_key =f'{citation_key}' # user manual input

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
page_contents=requests.get(url,headers=headers)
soup=BeautifulSoup(page_contents.text,'html.parser')

citation_dict={}
def cited_by_gscholar(dsoup):
    cited_by_link_list=[]
    for item in dsoup.find_all('div',{"class" : "gs_ri"}):
        cited_by_link_dict={}
        authors=[]
        for el in item.extract().find_all('a'):
            if 'https' in el.get('href').split(':'):
                if len(set(['View as HTML', 'Cached']) & set([str(el.text)])) ==0:
                    cited_by_link_dict['article_title']=el.text
                    cited_by_link_dict['article_link']=el.get('href')
            if '/citations?user' in el.get('href'):
                authors.append(el.text)
            elhref = el.get('href')
            if '/scholar?cites' in elhref.split('='):
                cited_by_link_dict['cited_by']=elhref
            if 'by' in el.text.split():
                #print(el.text.split())
                citation_cnt=re.findall(r'\d+', el.text)[0]
                cited_by_link_dict['citations_count']=citation_cnt
                cited_by_link_list.append(cited_by_link_dict)
        cited_by_link_dict['auhtors']=authors
    return cited_by_link_list

dsoup = copy.deepcopy(soup)
dsoup= cited_by_gscholar(dsoup)
citation_dict['citation_key']=citation_key
citation_dict['google_scholar_link']=url
citation_dict['cited_by']=dsoup
df = pd.DataFrame.from_dict(citation_dict)
df.to_csv('academic_citation_network.csv', mode='a')
citation_dict

article--1996--La-Porta-Rafael--Expectations-and-the-cross-section-of-stock-returns--The-Journal-of-Finance.pdf
CITATION KEY:  la1996expectations


{'citation_key': 'la1996expectations',
 'google_scholar_link': 'https://scholar.google.com/scholar?cites=15845218875244839334&as_sdt=5,47&sciodt=0,47&hl=en',
 'cited_by': [{'article_title': 'Human behavior and the efficiency of the financial system',
   'article_link': 'https://www.sciencedirect.com/science/article/pii/S1574004899100338',
   'cited_by': '/scholar?cites=10067409834998792250&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '1201',
   'auhtors': []},
  {'article_title': 'Asset pricing at the millennium',
   'article_link': 'https://onlinelibrary.wiley.com/doi/abs/10.1111/0022-1082.00260',
   'cited_by': '/scholar?cites=2583667990489829912&as_sdt=5,47&sciodt=0,47&hl=en',
   'citations_count': '1273',
   'auhtors': ['JY Campbell']},
  {'article_title': 'Replicating anomalies',
   'article_link': 'https://academic.oup.com/rfs/article-pdf/doi/10.1093/rfs/hhy131/33710468/hhy131.pdf',
   'cited_by': '/scholar?cites=3268850943037854099&as_sdt=5,47&sciodt=0,47&hl=en',
   'ci